In [61]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import time
import random
import warnings
import logging
import re
from nltk.tokenize import sent_tokenize
from collections import Counter

In [62]:
# Ensure NLTK data is available
import nltk
nltk.download('punkt', quiet=True)

# Set up logging for debugging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger(__name__)

# Suppress warnings (optional)
warnings.filterwarnings("ignore", category=FutureWarning)

# Load BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)  # Binary classification
model.eval()  # Set model to evaluation mode

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
# --- Step 1: Function to summarize content using generative AI ---
def summarize_content(statement, content, max_sentences=5, max_length=2000):
    """Summarize content using a generative AI model, focusing on information relevant to the statement."""
    try:
        # Construct a prompt for the generative model
        prompt = (
            f"Given the following statement: '{statement}'\n"
            f"And the following content: '{content[:10000]}'\n"
            f"Summarize the content in up to {max_sentences} sentences, including only information directly relevant to the statement. "
            f"Exclude irrelevant details and ensure the summary is coherent, concise, and avoids gibberish. "
            f"Limit the summary to {max_length} characters. If no relevant information is found, return 'No relevant summary found'."
        )

        # Placeholder for generative model API call (replace with your API or model)
        try:
            # Example: Simulating an API call to a generative model (e.g., xAI's Grok API)
            # Replace this with your actual API endpoint and authentication
            api_url = "https://x.ai/api"  # Placeholder; use actual API endpoint
            headers = {"Authorization": "Bearer YOUR_API_ID"}  # Replace with your API key
            payload = {
                "prompt": prompt,
                "max_tokens": 500,  # Adjust based on your model's token limit
                "temperature": 0.7,  # For coherent output
                "top_p": 0.9
            }
            response = requests.post(api_url, json=payload, headers=headers, timeout=10)
            response.raise_for_status()
            summary = response.json().get("summary", "No relevant summary found")  # Adjust based on API response structure

            # Validate and truncate summary
            if len(summary) > max_length:
                sentences = sent_tokenize(summary)
                truncated = ""
                current_length = 0
                for sentence in sentences:
                    if current_length + len(sentence) <= max_length:
                        truncated += sentence + " "
                        current_length += len(sentence) + 1
                    else:
                        break
                summary = truncated.strip() or "No relevant summary found"

            if not summary or summary == "No relevant summary found":
                logger.warning("Generative model returned empty or irrelevant summary")
                return fallback_summarize_content(statement, content, max_sentences, max_length)

            logger.info(f"Generated summary for statement '{statement[:60]}...': {summary[:100]}...")
            return summary

        except Exception as api_error:
            logger.error(f"Generative AI API error: {api_error}")
            # Fallback to heuristic-based summarization
            return fallback_summarize_content(statement, content, max_sentences, max_length)

    except Exception as e:
        logger.error(f"Error summarizing content: {e}")
        return fallback_summarize_content(statement, content, max_sentences, max_length)

# --- Fallback Heuristic-Based Summarization ---
def fallback_summarize_content(statement, content, max_sentences=5, max_length=2000):
    """Fallback heuristic-based summarization if generative AI fails."""
    try:
        # Tokenize content into sentences
        sentences = sent_tokenize(content)
        if not sentences:
            logger.warning("No sentences found in content")
            return "No relevant summary found"

        # Extract keywords from statement, excluding stop words
        stop_words = {'a', 'an', 'the', 'is', 'are', 'was', 'were', 'in', 'on', 'at', 'to', 'and', 'or', 'for', 'with', 'by', 'from', 'of'}
        statement_words = set(re.findall(r'\b\w+\b', statement.lower())) - stop_words
        if not statement_words:
            logger.warning("No meaningful keywords extracted from statement")
            return "No relevant summary found"

        # Score sentences based on keyword overlap and relevance
        scored_sentences = []
        for sentence in sentences:
            sentence_words = set(re.findall(r'\b\w+\b', sentence.lower()))
            if not sentence_words:
                continue
            overlap = len(statement_words.intersection(sentence_words))
            word_counts = Counter(sentence_words)
            repetition_score = sum(count > 2 for count in word_counts.values())
            sentence_length = len(sentence_words)
            is_valid = sentence_length > 3 and repetition_score == 0
            if overlap > 0 and is_valid:
                scored_sentences.append((sentence, overlap))

        # Sort by overlap score and select top sentences
        scored_sentences = sorted(scored_sentences, key=lambda x: x[1], reverse=True)[:max_sentences]
        summary_sentences = [s[0] for s in scored_sentences]

        if not summary_sentences:
            logger.warning("No relevant sentences found for summary")
            return "No relevant summary found"

        summary = " ".join(summary_sentences).strip()
        if len(summary) > max_length:
            truncated = ""
            current_length = 0
            for sentence in summary_sentences:
                if current_length + len(sentence) <= max_length:
                    truncated += sentence + " "
                    current_length += len(sentence) + 1
                else:
                    break
            summary = truncated.strip()

        if not summary:
            logger.warning("Summary is empty after processing")
            return "No relevant summary found"

        logger.info(f"Fallback summary for statement '{statement[:60]}...': {summary[:100]}...")
        return summary

    except Exception as e:
        logger.error(f"Error in fallback summarization: {e}")
        return "No relevant summary found"

In [64]:
# --- Step 2: Function to fetch content from a URL ---
def get_page_content(url):
    try:
        headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36"}
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, "html.parser")
        text_elements = soup.find_all(["p", "div", "article", "span"])
        text = " ".join([elem.get_text(strip=True) for elem in text_elements if elem.get_text(strip=True)])
        if not text:
            logger.warning(f"No content extracted from {url}")
            return "No content found"
        return text[:10000]  # Trim to 10000 chars
    except requests.exceptions.RequestException as e:
        logger.error(f"Error fetching {url}: {e}")
        return f"Error fetching content: {e}"
    except Exception as e:
        logger.error(f"Unexpected error for {url}: {e}")
        return f"Error fetching content: {e}"

In [65]:
# --- Step 3: Use googlesearch to get evidence URLs and content ---
def get_search_results(query, max_results=10):
    try:
        from googlesearch import search
        urls = list(search(query, num_results=max_results * 2))
        results = []
        skipped = 0
        for url in urls:
            if len(results) >= max_results:
                break
            if not url.startswith(("http://", "https://")):
                logger.warning(f"Skipping invalid URL: {url}")
                skipped += 1
                continue
            logger.info(f"Fetching content from {url}")
            content = get_page_content(url)
            if content.startswith("Error") or content == "No content found":
                logger.info(f"Skipping {url} due to {'error' if content.startswith('Error') else 'empty content'}")
                skipped += 1
            else:
                results.append({"url": url, "content": content})
            time.sleep(5)
        logger.info(f"Skipped {skipped} URLs, fetched {len(results)} valid results for query: {query[:60]}...")
        return results
    except Exception as e:
        logger.error(f"Search error for query '{query[:60]}...': {e}")
        return [{"url": None, "content": f"Search error: {e}"}]

In [66]:
# --- Step 4: Adjust prediction using BERT ---
def adjust_prediction_with_external_evidence(statement, evidence_data):
    """Use BERT to predict truthfulness based on statement and summarized evidence"""
    summaries = [
        summarize_content(statement, item["content"])
        for item in evidence_data
        if not item["content"].startswith("Error") and item["content"] != "No content found"
    ]

    if not summaries:
        logger.warning(f"No valid summaries found for statement: {statement[:60]}...")
        return 0.0, []  # Return 0.0 probability and empty scores list

    # Combine statement with each summarized evidence text
    probabilities = []
    for summary in summaries:
        if summary == "No relevant summary found":
            continue
        input_text = f"[CLS] {statement} [SEP] {summary} [SEP]"
        inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True, max_length=512)
        
        with torch.no_grad():
            outputs = model(**inputs)
            probs = torch.softmax(outputs.logits, dim=1).detach().numpy()[0]
            true_prob = float(probs[1])  # Probability of "true" class
            probabilities.append(true_prob)
    
    # Use max probability to avoid diluting strong signals
    final_prob = max(probabilities) if probabilities else 0.0
    logger.info(f"Adjusted probability for statement '{statement[:60]}...': {final_prob:.4f}")
    logger.info(f"Evidence probabilities: {probabilities}")
    return final_prob, probabilities

In [67]:
# --- Step 5: Apply adjustment over DataFrame ---
TEST_PATH = "liar_dataset/test.tsv"
try:
    test_df = pd.read_csv(TEST_PATH, sep='\t')
except FileNotFoundError:
    logger.error(f"File {TEST_PATH} not found.")
    exit(1)

# Verify column for statements
statement_col = 'statement' if 'statement' in test_df.columns else 2

# Sample 5 random rows
random_indices = random.sample(range(len(test_df)), 5)
sample_df = test_df.iloc[random_indices].copy()

# Initialize lists to store results
adjusted_probs = []
web_data = []

In [68]:
# Process sample
for counter, (i, row) in enumerate(sample_df.iterrows(), 1):
    statement = row[statement_col] if isinstance(statement_col, str) else row.iloc[statement_col]
    logger.info(f"Processing statement {counter}/{len(sample_df)}: {statement[:60]}...")
    
    # Fetch search results
    search_results = get_search_results(statement)
    
    # Calculate adjusted probability
    if search_results:
        adjusted_prob, evidence_probs = adjust_prediction_with_external_evidence(statement, search_results)
        print(f"Statement {counter}: {statement[:60]}...")
        print(f"Evidence probabilities: {evidence_probs}")
        adjusted_probs.append(adjusted_prob)
    else:
        logger.warning(f"No search results for statement: {statement[:60]}...")
        adjusted_probs.append(0.0)
        evidence_probs = []

    # Store web data with summaries and probabilities
    valid_result_index = 0
    for result in search_results:
        original_content = result["content"]
        summary = summarize_content(statement, original_content) if not original_content.startswith("Error") and original_content != "No content found" else original_content
        prob = evidence_probs[valid_result_index] if valid_result_index < len(evidence_probs) and summary != "No relevant summary found" else None
        web_data.append({
            "statement": statement,
            "url": result["url"],
            "content": original_content,
            "content_summary": summary,
            "probability": prob
        })
        if summary != "No relevant summary found" and not original_content.startswith("Error") and original_content != "No content found":
            valid_result_index += 1

# Create DataFrame for web data
web_df = pd.DataFrame(web_data)

# Define threshold
BEST_THRESHOLD = 0.6

# Update sample DataFrame with adjusted results
sample_df["Adjusted Final Probability"] = [f"{p:.4f}" for p in adjusted_probs]
sample_df["Adjusted Final Class"] = ['true' if p > BEST_THRESHOLD else 'false' for p in adjusted_probs]

2025-07-22 18:10:56,937 - INFO - Processing statement 1/5: Neville Chamberlain told the British people: Accept the Nazi...
2025-07-22 18:10:57,588 - WARNING - Skipping invalid URL: /search?num=22
2025-07-22 18:10:57,588 - INFO - Fetching content from https://www.politifact.com/factchecks/2013/oct/16/ted-cruz/ted-cruz-airs-popular-and-flawed-impression-britis/
2025-07-22 18:11:02,915 - INFO - Fetching content from https://www.brainyquote.com/quotes/ted_cruz_700187
2025-07-22 18:11:02,999 - ERROR - Error fetching https://www.brainyquote.com/quotes/ted_cruz_700187: 403 Client Error: Forbidden for url: https://www.brainyquote.com/quotes/ted_cruz_700187
2025-07-22 18:11:03,001 - INFO - Skipping https://www.brainyquote.com/quotes/ted_cruz_700187 due to error
2025-07-22 18:11:08,002 - INFO - Fetching content from https://www.bbc.com/news/magazine-24300094
2025-07-22 18:11:13,311 - INFO - Fetching content from https://www.statesman.com/story/news/2013/10/20/politifact-cruz-off-mark-on/67136860

Statement 1: Neville Chamberlain told the British people: Accept the Nazi...
Evidence probabilities: [0.6562915444374084, 0.7118706107139587, 0.7275639176368713, 0.7127876877784729, 0.7341047525405884, 0.6410230398178101, 0.7078063488006592, 0.7281387448310852, 0.6828721761703491]


2025-07-22 18:12:31,992 - ERROR - Generative AI API error: 403 Client Error: Forbidden for url: https://x.ai/api
2025-07-22 18:12:31,995 - INFO - Fallback summary for statement 'Neville Chamberlain told the British people: Accept the Nazi...': Yes, they will dominate the continent of Europe, but that is not our problem. Yes, they will dominat...
2025-07-22 18:12:32,037 - ERROR - Generative AI API error: 403 Client Error: Forbidden for url: https://x.ai/api
2025-07-22 18:12:32,040 - INFO - Fallback summary for statement 'Neville Chamberlain told the British people: Accept the Nazi...': Advocacy and Legislative AffairsEuropean UnionLatin AmericaIntercommunal AffairsSenior Advocacy & Ho...
2025-07-22 18:12:32,090 - ERROR - Generative AI API error: 403 Client Error: Forbidden for url: https://x.ai/api
2025-07-22 18:12:32,096 - INFO - Fallback summary for statement 'Neville Chamberlain told the British people: Accept the Nazi...': Yes, they will dominate the continent of Europe, but that is

Statement 2: Narragansett Bay waters are getting warmer -- 4 degrees Fahr...
Evidence probabilities: [0.7541549205780029, 0.8174713253974915, 0.7540768980979919, 0.7485733032226562, 0.7541549205780029, 0.7453793287277222, 0.6820431351661682, 0.7909998297691345]


2025-07-22 18:14:11,005 - ERROR - Generative AI API error: 403 Client Error: Forbidden for url: https://x.ai/api
2025-07-22 18:14:11,008 - INFO - Fallback summary for statement 'Narragansett Bay waters are getting warmer -- 4 degrees Fahr...': However, climate change and its warmer temperatures, can be linked directly or indirectly to many of...
2025-07-22 18:14:11,062 - ERROR - Generative AI API error: 403 Client Error: Forbidden for url: https://x.ai/api
2025-07-22 18:14:11,067 - INFO - Fallback summary for statement 'Narragansett Bay waters are getting warmer -- 4 degrees Fahr...': ����8�}�?|��Q��aNXWo���������Y��������^?����C[���
l���"��s\bX�:���Ѭfޯ�88��{�����bm?�0��CLpԶ?\��...
2025-07-22 18:14:11,120 - ERROR - Generative AI API error: 403 Client Error: Forbidden for url: https://x.ai/api
2025-07-22 18:14:11,125 - WARNING - Summary is empty after processing
2025-07-22 18:14:11,164 - ERROR - Generative AI API error: 403 Client Error: Forbidden for url: https://x.ai/api
2025-07

...

2025-07-22 18:15:26,451 - ERROR - Generative AI API error: 403 Client Error: Forbidden for url: https://x.ai/api
2025-07-22 18:15:26,454 - INFO - Fallback summary for statement 'The Milwaukee business community did not speak about the fac...': "They'll actually be in service -- serving people and providing a new transit option in another comm...
2025-07-22 18:15:26,540 - ERROR - Generative AI API error: 403 Client Error: Forbidden for url: https://x.ai/api
2025-07-22 18:15:26,547 - INFO - Fallback summary for statement 'The Milwaukee business community did not speak about the fac...': "Talgo is also disappointed that the business community did not speak about the facts on this projec...
2025-07-22 18:15:26,590 - ERROR - Generative AI API error: 403 Client Error: Forbidden for url: https://x.ai/api
2025-07-22 18:15:26,596 - INFO - Fallback summary for statement 'The Milwaukee business community did not speak about the fac...': -Become a Member -Member LoginShow MenuReal EstateListingsAd

Statement 4: This March, for the first time in human history, the monthly...
Evidence probabilities: [0.686287522315979, 0.7013114094734192, 0.8070313334465027, 0.7555510401725769, 0.79030841588974, 0.6958007216453552, 0.7968891859054565]


2025-07-22 18:16:38,653 - ERROR - Generative AI API error: 403 Client Error: Forbidden for url: https://x.ai/api
2025-07-22 18:16:38,658 - INFO - Fallback summary for statement 'This March, for the first time in human history, the monthly...': ToolsCoastal Risk FinderGlobal Climate Shift IndexFloodVision RiskViewer Tools ImpactStoriesResearch...
2025-07-22 18:16:38,697 - ERROR - Generative AI API error: 403 Client Error: Forbidden for url: https://x.ai/api
2025-07-22 18:16:38,702 - INFO - Fallback summary for statement 'This March, for the first time in human history, the monthly...': By drilling more than3 kilometers deep into the ice sheets over Greenland and Antarctica, scientists...
2025-07-22 18:16:38,742 - ERROR - Generative AI API error: 403 Client Error: Forbidden for url: https://x.ai/api
2025-07-22 18:16:38,748 - INFO - Fallback summary for statement 'This March, for the first time in human history, the monthly...': And in September 2016, the usual annual low skimmed above 40

Statement 5: In many instances, (peoples) health care costs are more than...
Evidence probabilities: [0.7236899137496948, 0.7160677313804626, 0.7632182240486145, 0.7055948972702026, 0.743159294128418, 0.6464502215385437, 0.7958265542984009, 0.6982211470603943]


2025-07-22 18:18:23,224 - ERROR - Generative AI API error: 403 Client Error: Forbidden for url: https://x.ai/api
2025-07-22 18:18:23,225 - WARNING - Summary is empty after processing
2025-07-22 18:18:23,260 - ERROR - Generative AI API error: 403 Client Error: Forbidden for url: https://x.ai/api
2025-07-22 18:18:23,262 - INFO - Fallback summary for statement 'In many instances, (peoples) health care costs are more than...': "It's like you're being punished for being sick," Wingard said.What's Broken:Unpaid medical bills ca...
2025-07-22 18:18:23,295 - ERROR - Generative AI API error: 403 Client Error: Forbidden for url: https://x.ai/api
2025-07-22 18:18:23,301 - INFO - Fallback summary for statement 'In many instances, (peoples) health care costs are more than...': Only Miami—which has much higher labor and living costs—spends more per person on health care. In ot...
2025-07-22 18:18:23,338 - ERROR - Generative AI API error: 403 Client Error: Forbidden for url: https://x.ai/api
2025-07-

In [69]:
sample_df

,11972.json,true,Building a wall on the U.S.-Mexico border will take literally years.,immigration,rick-perry,Governor,Texas,republican,30,30.1,42,23,18,Radio interview,Adjusted Final Probability,Adjusted Final Class
122,8406.json,barely-true,Neville Chamberlain told the British people: A...,"health-care,history",ted-cruz,Senator,Texas,republican,36,33,15,19,8,speech advocating repeal of Obamacare law,0.7341,true
620,7706.json,half-true,Narragansett Bay waters are getting warmer -- ...,"climate-change,environment,history,science",sheldon-whitehouse,U.S. Senator,Rhode Island,democrat,2,3,2,7,0,a Senate speech published on YouTube,0.8175,true
789,2986.json,true,The Milwaukee business community did not speak...,"state-budget,transportation",talgo-inc,Spanish-owned manufacturer of trains,NaN,none,0,0,0,0,0,a statement from the company,0.7381,true
1056,10962.json,true,"This March, for the first time in human histor...","climate-change,congress",sheldon-whitehouse,U.S. Senator,Rhode Island,democrat,2,3,2,7,0,U.S. Senate,0.8070,true
1009,13409.json,barely-true,"In many instances, (peoples) health care costs...","economy,government-regulation,health-care,hist...",donald-trump,President-Elect,New York,republican,63,114,51,37,61,a speech,0.7958,true


In [70]:
web_df

,statement,url,content,content_summary,probability
0,Neville Chamberlain told the British people: A...,https://www.politifact.com/factchecks/2013/oct...,The Poynter InstituteMenuDonateState EditionsC...,Chamberlain responded with a British declarati...,0.656292
1,Neville Chamberlain told the British people: A...,https://www.bbc.com/news/magazine-24300094,BBC HomepageSkip to contentAccessibility HelpY...,"Yes, they'll dominate the continent of Europe ...",0.711871
2,Neville Chamberlain told the British people: A...,https://amymantravadi.com/2013/10/01/lets-give...,"Amy Mantravadi""Never underestimate what a woma...","""About This SiteThe Chronicle of Maud(A)Millen...",0.727564
3,Neville Chamberlain told the British people: A...,https://www.lapresse.ca/international/201805/0...,"Consulter lapresse.ca La Presse, mardi 22 juil...","Yes, they'll dominate the continent of Europe ...",0.712788
4,Neville Chamberlain told the British people: A...,https://frasersherman.com/2013/09/25/oh-good-g...,Fraser Sherman's BlogSkip to contentHomeAbout ...,"Yes, they will dominate the continent of Europ...",0.734105
5,Neville Chamberlain told the British people: A...,https://www.bnaibrith.org/sen-ted-cruzs-use-of...,AboutMissionAround the WorldOur Leadership2025...,Advocacy and Legislative AffairsEuropean Union...,0.641023
6,Neville Chamberlain told the British people: A...,https://www.econlib.org/archives/2013/09/hende...,Liberty Fund NetworkEconlibLiberty FundOLLAdam...,"Yes, they will dominate the continent of Europ...",0.707806
7,Neville Chamberlain told the British people: A...,https://whyy.org/articles/ted-cruz-uses-the-n-...,Skip to contentStudio 2Listen LiveListen LiveN...,"Yes, they’ll dominate the continent of Europe,...",0.728139
8,Neville Chamberlain told the British people: A...,https://www.quora.com/Why-is-Neville-Chamberla...,Something went wrong. Wait a moment and try ag...,No relevant summary found,NaN
9,Neville Chamberlain told the British people: A...,https://www.nationalww2museum.org/war/articles...,VisitStayEventsMoreTravelStoreCampusCalendarBB...,During a discussion of the ongoing diplomatic ...,0.682872


In [72]:
web_df['content'][0]

'The Poynter InstituteMenuDonateState EditionsCaliforniaFloridaIowaMichiganNew HampshireNew YorkNorth CarolinaPennsylvaniaTexasWest VirginiaWisconsinIssuesAll IssuesOnline hoaxesCoronavirusHealth CareImmigrationExtremismTaxesMarijuanaEnvironmentCrimeGunsForeign PolicyLGBTQ+PeopleDonald TrumpJD VanceMitch McConnellMike JohnsonJoe BidenMediaPunditFactTucker CarlsonSean HannityRachel MaddowBloggersPolitiFact VideosCampaigns2024 ElectionsTruth-o-MeterTrueMostly TrueHalf TrueMostly FalseFalsePants on FirePromisesMAGA-MeterBiden Promise TrackerTrump-O-MeterObameterLatest PromisesAbout UsOur ProcessOur StaffEn EspañolWho pays for PolitiFact?Advertise with UsSuggest a Fact-checkCorrections and UpdatesNewslettersDonateFollow usThe Facts NewsletterSign upEnglishEspañolStand up for the facts!Our only agenda is to publish the truth so you can be an informed participant in democracy.We need your help.More InfoI would like to contributeOne TimeMonthlyYearlyJoin NowTed Cruzstated on September 24, 201

In [73]:
web_df['content_summary'][0]

'Chamberlain responded with a British declaration of war on Germany.Historians say it didn\'t happenStill, four historians (including three based in England) uniformly told us Cruz’s claim was inaccurate in that it’s not correct that Chamberlain told the British people to accept Nazi domination of the continent because Britain couldn’t possibly stand against them.Cruz’s statement is "skewed,"William Roger Louis, a University of Texas history professor whose specialties include the history and politics of 20th century Britain, said by telephone. Yes, they will dominate the continent of Europe, but that is not our problem. Yes, they will dominate the continent of Europe, but that is not our problem. The Poynter InstituteMenuDonateState EditionsCaliforniaFloridaIowaMichiganNew HampshireNew YorkNorth CarolinaPennsylvaniaTexasWest VirginiaWisconsinIssuesAll IssuesOnline hoaxesCoronavirusHealth CareImmigrationExtremismTaxesMarijuanaEnvironmentCrimeGunsForeign PolicyLGBTQ+PeopleDonald TrumpJD

In [74]:
web_df['statement'][0]

'Neville Chamberlain told the British people: Accept the Nazis. Yes, they will dominate the continent of Europe, but that is not our problem. Lets appease them. Why? Because it cant be done. We cannot possibly stand against them.'